In [92]:
import numpy as np
import face_detection
import os, python_speech_features
import scipy.io.wavfile as wav
import pandas as pd
import torch.nn.functional as F
from talkNet import talkNet
from dataset import MyDataset
from torch.utils.data import DataLoader
import tqdm
import torch
import cv2
import tools
import subprocess
detector = face_detection.build_detector(
"DSFDDetector", confidence_threshold=.3, nms_iou_threshold=.3)

In [112]:
inputVideo =  "C:/Users/jmmol/Desktop/LIP-RTVE/MP4s/speaker212/speaker212_0010.mp4"
videoDuration = tools.checkVideoDuration(inputVideo)
tools.splitVideo(inputVideo,2,videoDuration)

speaker212_0010.mp4


In [94]:
#Carga vídeo
res, facePos = tools.saveFaceCrops(inputVideo,detector)
res[0].shape

(112, 112)

In [95]:
# AUDIO PROCESSING
audioPath = tools.convert_video_to_audio_ffmpeg(inputVideo)
_,sig = wav.read(audioPath)#r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker042_0063.wav") #
# _,sig = wav.read(r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav") 
# audioPath = r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav"
audio = python_speech_features.mfcc(sig, 16000, numcep = 13, winlen = 0.025, winstep = 0.010) #ASUME VIDEO A 25 Y AUDIO A 100, MODIFICAR


#TEST
#audio = torch.FloatTensor(audio[:180]).unsqueeze(0)
#video = torch.FloatTensor(res[:45]).unsqueeze(0)


In [101]:
model = talkNet()
model.load_state_dict(torch.load("./exps/exp1/model/model21_0006.model"))

05-24 13:46:28 Model para number = 15.01


<All keys matched successfully>

In [102]:
videoDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/npz"
audioDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/mfccs"
datasetTrain = MyDataset(51,videoDir,audioDir,"testSamples.csv")
item = datasetTrain.__getitem__(1050)
valLoader = DataLoader(dataset=datasetTrain,shuffle=False,batch_size=32,num_workers=14) #Cambiar num_workers

25 100


In [7]:
correctFrames, totalFrames = 0, 0
correctSamples, totalSamples = 0, 0
totalPreds = []
for num, (audioFeature, visualFeature, labels) in enumerate(tqdm.tqdm(valLoader)):
    with torch.no_grad():    
        output = model((audioFeature,visualFeature))
        labels = labels.cuda()
        batchPreds = torch.reshape(output[1], labels.shape)
        #Precision a nivel de video
        videoPreds = torch.mode(batchPreds,dim=1)[0]
        totalPreds.extend(videoPreds.detach().cpu().numpy().astype(int))
        labelMode = torch.mode(labels,dim=1)[0]
        correctSamples += (videoPreds == labelMode).sum().float()
        totalSamples += len(labels)
        # Precision a nivel de frame
        labels = labels.reshape((-1))
        correctFrames += (output[1] == labels).sum().float()
        totalFrames += len(labels) 
#7865
print("Frame acc:", correctFrames/totalFrames)
print("Sample acc:", correctSamples/totalSamples)

  0%|          | 0/938 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
print("Audios iguales?",(audio == item[0]).sum(),audio.shape[0]*audio.shape[1])
print("Videos iguales?",(video == item[1]).sum(),video.shape[0]*112*112)
print(audio[0],item[0][0])

Audios iguales? tensor(2652) 2652
Videos iguales? tensor(328442) 639744
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [103]:
predArray = []
for i in range(len(res)):
    print("FRAME Nº ",i)
    center = i
    iAudio = tools.padAudio(audio,1,center,21,len(res)).unsqueeze(0)
    iVideo = tools.padVideo(res,center,21).unsqueeze(0)
    #print("audio shape: ", audio.shape)
    #print("video shape: ", video.shape)
    # iAudio = item[0].unsqueeze(0)
    # iVideo = item[1].unsqueeze(0)
    output = model((iAudio,iVideo))
    #output = model((audio,video))
    #print(output)
    print("Pred ->",torch.mode(output[1])[0])
    print("Prob Pos ->",torch.mean(output[1]))
    predArray.append(torch.mean(output[1]).detach().cpu().numpy())

FRAME Nº  0
Pred -> tensor(0., device='cuda:0')
Prob Pos -> tensor(0., device='cuda:0')
FRAME Nº  1
Pred -> tensor(0., device='cuda:0')
Prob Pos -> tensor(0., device='cuda:0')
FRAME Nº  2
Pred -> tensor(0., device='cuda:0')
Prob Pos -> tensor(0., device='cuda:0')
FRAME Nº  3
Pred -> tensor(0., device='cuda:0')
Prob Pos -> tensor(0., device='cuda:0')
FRAME Nº  4
Pred -> tensor(1., device='cuda:0')
Prob Pos -> tensor(0.8571, device='cuda:0')
FRAME Nº  5
Pred -> tensor(1., device='cuda:0')
Prob Pos -> tensor(1., device='cuda:0')
FRAME Nº  6
Pred -> tensor(1., device='cuda:0')
Prob Pos -> tensor(1., device='cuda:0')
FRAME Nº  7
Pred -> tensor(1., device='cuda:0')
Prob Pos -> tensor(1., device='cuda:0')
FRAME Nº  8
Pred -> tensor(1., device='cuda:0')
Prob Pos -> tensor(1., device='cuda:0')
FRAME Nº  9
Pred -> tensor(1., device='cuda:0')
Prob Pos -> tensor(1., device='cuda:0')
FRAME Nº  10
Pred -> tensor(1., device='cuda:0')
Prob Pos -> tensor(0.8095, device='cuda:0')
FRAME Nº  11
Pred -> te

In [80]:
print(len(predArray))

183


In [10]:
center = 24
iAudio = tools.padAudio(audio,1,center,51,len(res)).unsqueeze(0)
iVideo = tools.padVideo(res,center,51).unsqueeze(0)
#print("audio shape: ", audio.shape)
#print("video shape: ", video.shape)

# iAudio = item[0].unsqueeze(0)
# iVideo = item[1].unsqueeze(0)
output = model((iAudio,iVideo))
#output = model((audio,video))
print(output)
print("Pred ->",torch.mode(output[1])[0])
print("Prob Pos ->",torch.mean(output[1]))

(tensor([[6.9808e-04, 9.9930e-01],
        [3.5927e-04, 9.9964e-01],
        [4.1301e-04, 9.9959e-01],
        [4.1630e-04, 9.9958e-01],
        [3.5234e-04, 9.9965e-01],
        [4.6210e-04, 9.9954e-01],
        [4.6554e-04, 9.9953e-01],
        [2.9334e-04, 9.9971e-01],
        [7.8696e-04, 9.9921e-01],
        [6.1939e-04, 9.9938e-01],
        [4.0660e-04, 9.9959e-01],
        [7.2922e-04, 9.9927e-01],
        [2.8408e-04, 9.9972e-01],
        [2.7339e-04, 9.9973e-01],
        [2.9624e-04, 9.9970e-01],
        [3.5977e-04, 9.9964e-01],
        [3.2451e-04, 9.9968e-01],
        [3.3486e-04, 9.9967e-01],
        [2.6592e-03, 9.9734e-01],
        [2.9554e-04, 9.9970e-01],
        [3.5991e-04, 9.9964e-01],
        [5.8302e-04, 9.9942e-01],
        [4.7461e-04, 9.9953e-01],
        [4.3599e-04, 9.9956e-01],
        [3.6730e-04, 9.9963e-01],
        [2.4271e-04, 9.9976e-01],
        [1.5686e-03, 9.9843e-01],
        [4.2929e-04, 9.9957e-01],
        [3.6547e-04, 9.9963e-01],
        [4.50

In [115]:
def createVideo(imgFrames,audioPath,width,height):
    video = cv2.VideoWriter("output.mp4", 0, 25, (width,height))

    for image in imgFrames:
        video.write(image)

    cv2.destroyAllWindows()
    video.release()
    print(' '.join(["ffmpeg","-y","-i",os.getcwd()+"/output.mp4","-i",audioPath,"-map","0:v","-map","1:a", "-c:v", "copy", "-shortest", os.getcwd()+"/output2.mp4"]))
    res = subprocess.check_output(["ffmpeg","-y","-i",os.getcwd()+f"/output.mp4","-i",audioPath,"-map","0:v","-map","1:a", "-c:v", "copy", "-shortest", os.getcwd()+f"/outputs/{inputVideo[-19:]}"])
    os.remove(os.getcwd()+f"/output.mp4")
    
    print(res)

In [116]:
i=0
cap = cv2.VideoCapture(inputVideo)
videoImages = []
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(predArray)
while i < len(predArray):
    ret, image = cap.read()
    greenValue = int(255*predArray[i])
    redValue = 255-greenValue
    color = (0,greenValue,redValue)
    #print(color)
    
    xmin,ymin,xmax,ymax = facePos[i]
    image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax),color , 1)
    cv2.putText(image, str(predArray[i]), (xmin, ymin-5), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (36,255,12), 1)
    videoImages.append(image)

    # cv2.imshow("My Video", image)
    # cv2.waitKey(0)
    i+=1
    if ret == False:
        break

cap.release()
cv2.destroyAllWindows()

createVideo(videoImages,audioPath,width,height)

[array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0.85714287, dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(0.8095238, dtype=float32), array(0.0952381, dtype=float32), array(0.1904762, dtype=float32), array(0.0952381, dtype=float32), array(0.33333334, dtype=float32), array(0.14285715, dtype=float32), array(0.33333334, dtype=float32), array(0.52380955, dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0.0952381, dtype=float32), array(0.52380955, dtype=float32), array(0.5714286, dtype=float32), array(0.8095238, dtype=float32), array(1., dtype=float32), array(0.8095238, dtype=float32), array(0.7619048, dtype=float32), array(0.71428573, dtype=float32), array(0.47619048, dtype=float32), array(0.523809

In [ ]:
#print(totalPreds)
dfPreds = pd.DataFrame(totalPreds,columns=["Prediction"])
dfSamples = pd.read_csv("testSamples.csv")
dfRes = dfSamples.join(dfPreds)
print(dfRes)
dfRes.to_csv("testPreds5.csv")

       Unnamed: 0            video            audio  label  center  Prediction
0               0  speaker073_0000  speaker073_0000      1      34           1
1               1  speaker205_0000  speaker205_0001      0      75           0
2               2  speaker163_0000  speaker163_0000      1     118           1
3               3  speaker314_0011  speaker314_0011      1      18           1
4               4  speaker300_0001  speaker300_0001      0      19           0
...           ...              ...              ...    ...     ...         ...
29996       29996  speaker310_0006  speaker310_0006      1      59           1
29997       29997  speaker169_0002  speaker169_0002      1      50           1
29998       29998  speaker291_0000  speaker155_0006      0     143           0
29999       29999  speaker258_0000  speaker197_0005      0      96           0
30000       30000  speaker065_0025  speaker065_0025      1      25           1

[30001 rows x 6 columns]


In [ ]:
dfTrain = pd.read_csv("trainSamples.csv")

In [ ]:
dfTrain.join(dfSamples,on="video")

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat